In [1]:
from crowd_evaluation import ConfidenceEvaluatorC
from datasets import BicyclesDataset
import numpy as np
from tqdm.notebook import tqdm

In [2]:
dataset = BicyclesDataset('/Users/julianassmann/Documents/Bachelor-Thesis/Code/datasets/bicycles/bicycles_with_truth.csv')

In [3]:
p_measured = dataset.get_measured_error_rates_for_workers(dataset.workers)

from joblib import Memory, Parallel, delayed
memory = Memory('bicycles_confidence_levels', verbose=0)
@memory.cache
def calc_accuracies_and_interval_size(
        confidence: float,
        evaluator_name: str):
    evaluator = ConfidenceEvaluatorC(dataset)
    ps, confs = evaluator.evaluate_workers_with_confidence(dataset.workers, confidence=confidence)
    min_limit, max_limit = ps-confs, ps+confs

    inside_interval = np.where((min_limit <= p_measured) & (p_measured <= max_limit), 1, 0)
    accuracy = np.count_nonzero(inside_interval) / dataset.num_workers
    return accuracy, np.average(2*confs)


confidence_levels = np.arange(0, 1, 0.05)
accuracies = []
interval_sizes = dict()

res = Parallel(n_jobs=7)(delayed(calc_accuracies_and_interval_size)(c, 'c') for c in tqdm(confidence_levels))

print(res)
print('\a')

  0%|          | 0/20 [00:00<?, ?it/s]

[(0.0, 0.0), (0.0, 0.00045012633), (0.0, 0.0009020313), (0.0, 0.0013575435), (0.0, 0.0018185944), (0.0, 0.00228728), (0.0, 0.0027659351), (0.0, 0.0032572285), (0.0, 0.0037642897), (0.0, 0.0042908844), (0.0, 0.004841672), (0.0, 0.0054225745), (0.0, 0.0060413857), (0.0, 0.0067087356), (0.0, 0.0074398), (0.0, 0.00825752), (0.0, 0.009199325), (0.0, 0.010333348), (0.0, 0.011807207), (0.0, 0.014069155)]

